In [1]:
!pip install transformers


In [3]:
!pip install pyspellchecker


In [11]:
### This is the file for Fine tuning

import pandas as pd
import re
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Step 1: Load and clean the dataset
def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    return s

# Load dataset
df = pd.read_csv('/content/cleaned_fitness_qa_dataset.csv')
# Update the column names in the cleaning step
df['question'] = df['question'].apply(cleaning)
df['answer'] = df['answer'].apply(cleaning)

# Combine Question and Answer for training
df['QA'] = "Question: " + df['question'] + " Answer: " + df['answer']

# Save the cleaned dataset to a text file
train_file_path = "/content/fitness_qa_train.txt"
with open(train_file_path, 'w') as f:
    f.write("\n".join(df['QA']))


# Step 2: Define functions for loading dataset and fine-tuning GPT-2
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

def load_data_collator(tokenizer, mlm=False):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm
    )

# Step 3: Fine-tune GPT-2
model_name = 'gpt2'
output_dir = '/content/fitness_gpt2_model'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 requires a padding token

# Load dataset and collator
train_dataset = load_dataset(train_file_path, tokenizer)
data_collator = load_data_collator(tokenizer)

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained(model_name)

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_dir='/content/logs',
    logging_steps=100,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

# Train the model
trainer.train()
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

# Step 4: Define text generation function
def generate_text(sequence, max_length, model_path, tokenizer_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model
model_path = output_dir
tokenizer_path = output_dir
sequence = "How do I build muscle?"
max_length = 50
response = generate_text(sequence, max_length, model_path, tokenizer_path)
print("Generated Response:", response)
###

In [6]:
#import statements
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import requests
from spacy.pipeline import EntityRuler
from typing import List, Dict, Tuple
import time
import spacy
from spellchecker import SpellChecker
import re
import logging
from nltk.corpus import stopwords
import nltk

# Download NLTK stop words (if not already downloaded)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

logging.getLogger("transformers").setLevel(logging.ERROR)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **NER for `FOOD`**

**NER using Entity ruler for the identification of food items**

In [12]:
#Adding food label to the following items in the Entity Ruler
def add_fooditems(nlp):

    # Add custom food entities to the pipeline
    food_items = [
    # Meats and Fish
    "Chicken", "Mutton", "Pork", "Chicken Breast", "Chicken Thigh", "Chicken Wings", "Turkey Breast",
    "Ground Turkey", "Beef", "Lean Beef", "Ground Beef", "Beef Steak", "Pork Chop", "Pork Tenderloin",
    "Bacon", "Ham", "Salmon Fillet", "Tuna", "Tilapia", "Cod", "Shrimp", "Sardines", "Lamb Chop",

    # Eggs and Dairy
    "Whole Egg", "Egg White", "Whole Milk", "Skim Milk", "Greek Yogurt", "Regular Yogurt", "Cottage Cheese",
    "Cheddar Cheese", "Mozzarella Cheese", "String Cheese", "Whipped Cream", "Butter",

    # Plant Based Proteins
    "Tofu", "Tempeh", "Seitan", "Black Beans", "Kidney Beans", "Chickpeas", "Lentils", "Edamame", "Black Eyed Peas",
    "Quinoa",

    # Grains and Starches
    "White Rice", "Brown Rice", "Jasmine Rice", "White Bread", "Whole Wheat Bread", "Sourdough Bread", "Bagel",
    "English Muffin", "Tortilla", "Pita Bread", "Oatmeal", "Steel Cut Oats", "Cream of Wheat", "Pasta",
    "Whole Wheat Pasta", "Sweet Potato", "White Potato", "Corn",

    # Vegetables
    "Broccoli", "Spinach", "Kale", "Lettuce", "Cucumber", "Tomato", "Bell Pepper", "Carrots", "Green Beans",
    "Asparagus", "Brussels Sprouts", "Cauliflower", "Zucchini", "Eggplant", "Mushrooms", "Onion", "Garlic", "Celery",
    "Beets", "Artichoke",

    # Fruits
    "Apple", "Banana", "Orange", "Strawberries", "Blueberries", "Raspberries", "Blackberries", "Grapes",
    "Pineapple", "Mango", "Pear", "Peach", "Plum", "Watermelon", "Cantaloupe", "Honeydew", "Kiwi", "Pomegranate",
    "Grapefruit", "Lemon", "Lime",

    # Nuts and Seeds
    "Almonds", "Walnuts", "Cashews", "Pistachios", "Pecans", "Brazil Nuts", "Macadamia Nuts", "Peanuts",
    "Chia Seeds", "Flax Seeds", "Sunflower Seeds", "Pumpkin Seeds", "Hemp Seeds",

    # Healthy Fats
    "Avocado", "Olive Oil", "Coconut Oil", "Avocado Oil", "Peanut Butter", "Almond Butter", "Cashew Butter", "Tahini",

    # Supplements
    "Whey Protein", "Casein Protein", "Plant Protein", "Mass Gainer", "Protein Bar", "Energy Bar", "Pre Workout",
    "Creatine", "BCAA",

    # Processed and Packaged Foods
    "Protein Cookie", "Protein Chips", "Rice Cakes", "Crackers", "Granola", "Cereal", "Protein Cereal", "Trail Mix",
    "Dried Fruit", "Beef Jerky", "Turkey Jerky", "Protein Shake", "Smoothie",

    # Condiments and Sauces
    "Hummus", "Mustard", "Hot Sauce", "Salsa", "Guacamole", "Greek Yogurt Dip", "Protein Spread", "Sugar Free Syrup",
    "Low Fat Dressing",

    # Beverages
    "Black Coffee", "Green Tea", "Black Tea", "Protein Coffee", "Sports Drink", "Energy Drink", "Coconut Water",
    "Almond Milk", "Soy Milk", "Oat Milk", "Protein Water"
]
    ruler = nlp.add_pipe("entity_ruler", before="ner")
    patterns = [{"label": "FOOD", "pattern": item.lower()} for item in food_items]
    ruler.add_patterns(patterns)



In [14]:
#method to get the nutrition
def get_calorie(items):
    api_url = 'https://api.calorieninjas.com/v1/nutrition?query='
    for item in items:
        query = item
        response = requests.get(api_url + query, headers={'X-Api-Key': '41XzPlIpAcWTDGPLoj6WAA==NcwVJXSV0o0BEeJ3'})
        if response.status_code == requests.codes.ok:
            return response.json()
        else:
            return f"Error: {response.status_code}, {response.text}"

In [22]:
#modifying the information obtained by the nutrition API in a readable format
def print_nutritional_info(data):
    for item in data["items"]:
        paragraph = f"""The item is a {item['name']}. A 100g serving contains: {item['calories']} calories, {item['fat_total_g']}g of total fat, including {item['fat_saturated_g']}g of saturated fat, {item['protein_g']}g of protein {item['sodium_mg']}mg of sodium, {item['potassium_mg']}mg of potassium, {item['cholesterol_mg']}mg of cholesterol, {item['carbohydrates_total_g']}g of total carbohydrates, with {item['fiber_g']}g of fiber and {item['sugar_g']}g of sugar"""
    return paragraph

**Main method to identify the food items and get the nutrition information**

In [18]:
def nutrition(nlp,text):
    #NER identification\
    doc = nlp(text.lower())
    items = [ent.text for ent in doc.ents if ent.label_ == "FOOD"]
    if(items):
        data = get_calorie(items)
        if(data):
            return(print_nutritional_info(data))
        else:
            return None
    else :
        return None
        
    

# **NER FOR `BODY_PART`**

In [25]:
#NER MODEL using Entity Ruler method

def create_fitness_body_parts_ner(patterns: List[Dict[str, str]] = None) -> spacy.language.Language:
    """
    Create a custom NER pipeline for detecting fitness-related body parts.

    Args:
        patterns: Optional list of pattern dictionaries to override default patterns

    Returns:
        nlp: Configured spaCy pipeline with fitness body parts EntityRuler
    """
    nlp = spacy.blank("en")

    if patterns is None:
        patterns = [
    {"label": "BODY_PART", "pattern": "biceps"},
    {"label": "BODY_PART", "pattern": "triceps"},
    {"label": "BODY_PART", "pattern": "deltoids"},
    {"label": "BODY_PART", "pattern": "delts"},
    {"label": "BODY_PART", "pattern": "shoulders"},
    {"label": "BODY_PART", "pattern": "pectorals"},
    {"label": "BODY_PART", "pattern": "pecs"},
    {"label": "BODY_PART", "pattern": "chest"},
    {"label": "BODY_PART", "pattern": "upper chest"},
    {"label": "BODY_PART", "pattern": "lower chest"},
    {"label": "BODY_PART", "pattern": "abs"},
    {"label": "BODY_PART", "pattern": "abdominals"},
    {"label": "BODY_PART", "pattern": "upper abs"},
    {"label": "BODY_PART", "pattern": "lower abs"},
    {"label": "BODY_PART", "pattern": "obliques"},
    {"label": "BODY_PART", "pattern": "core"},
    {"label": "BODY_PART", "pattern": "quadriceps"},
    {"label": "BODY_PART", "pattern": "quads"},
    {"label": "BODY_PART", "pattern": "hamstrings"},
    {"label": "BODY_PART", "pattern": "calves"},
    {"label": "BODY_PART", "pattern": "glutes"},
    {"label": "BODY_PART", "pattern": "lats"},
    {"label": "BODY_PART", "pattern": "traps"},
    {"label": "BODY_PART", "pattern": "back"},
    {"label": "BODY_PART", "pattern": "upper back"},
    {"label": "BODY_PART", "pattern": "lower back"},
    {"label": "BODY_PART", "pattern": "mid back"},
    {"label": "BODY_PART", "pattern": "neck"},
    {"label": "BODY_PART", "pattern": "arms"},
    {"label": "BODY_PART", "pattern": "forearms"},
    {"label": "BODY_PART", "pattern": "legs"},
    {"label": "BODY_PART", "pattern": "thighs"},
    {"label": "BODY_PART", "pattern": "knees"},
    {"label": "BODY_PART", "pattern": "ankles"},
    {"label": "BODY_PART", "pattern": "upper body"},
    {"label": "BODY_PART", "pattern": "lower body"},
    {"label": "BODY_PART", "pattern": "midsection"},
    {"label": "BODY_PART", "pattern": "waist"},
    {"label": "BODY_PART", "pattern": "hips"},
    {"label": "BODY_PART", "pattern": "shoulder"},
    {"label": "BODY_PART", "pattern": "trapezius"},
    {"label": "BODY_PART", "pattern": "inner thighs"},
    {"label": "BODY_PART", "pattern": "outer thighs"},
    {"label": "BODY_PART", "pattern": "hands"},
    {"label": "BODY_PART", "pattern": "fingers"},
    {"label": "BODY_PART", "pattern": "wrists"},
    {"label": "BODY_PART", "pattern": "thumbs"},
    {"label": "BODY_PART", "pattern": "feet"},
    {"label": "BODY_PART", "pattern": "toes"},
    {"label": "BODY_PART", "pattern": "elbows"},
    {"label": "BODY_PART", "pattern": "shoulder joints"},
    {"label": "BODY_PART", "pattern": "hip joints"},
    {"label": "BODY_PART", "pattern": "wrist joints"},
    {"label": "BODY_PART", "pattern": "calf muscles"},
    {"label": "BODY_PART", "pattern": "forearm muscles"},
    {"label": "BODY_PART", "pattern": "lower legs"},
    {"label": "BODY_PART", "pattern": "upper legs"},
]


    # Create the EntityRuler and add it to the pipeline
    ruler = nlp.add_pipe("entity_ruler", config={"overwrite_ents": True})

    # Add the patterns to the ruler
    ruler.add_patterns(patterns)

    return nlp

In [27]:
#method to identify the BODY_PARTS
def extract_body_parts(text: str, nlp: spacy.language.Language) -> List[Tuple[str, int, int]]:
    """
    Extract fitness-related body parts from text.

    Args:
        text: Input text to process
        nlp: Configured spaCy pipeline with body parts EntityRuler

    Returns:
        List of tuples containing (body_part, start_char, end_char)
    """
    doc = nlp(text)
    return [(ent.text, ent.start_char, ent.end_char) for ent in doc.ents if ent.label_ == "BODY_PART"]


In [29]:
# Initialize the spell checker
spell = SpellChecker()

def autocorrect_sentence(sentence):
    # Tokenize the sentence into words
    words = sentence.split()

    # Correct each word
    corrected_words = []
    for word in words:
        if word.lower() in spell:  # Check if the word is valid
            corrected_words.append(word)  # Keep the word as is
        else:
            corrected_word = spell.correction(word)  # Suggest a correction
            corrected_words.append(corrected_word or word)  # Use correction if available

    # Join the corrected words back into a sentence
    corrected_sentence = ' '.join(corrected_words)
    return corrected_sentence

In [31]:
  def get_muscle_id():
      url = "https://wger.de/api/v2/muscle/"

      # Fetch data from the API
      response = requests.get(url)

      # Check if the request was successful
      if response.status_code == 200:
          data = response.json()
          muscles = data["results"]
          muscle_dict = {muscle["name_en"]: muscle["id"] for muscle in muscles}
          return muscle_dict
      else:
          print("Failed to retrieve data:", response.status_code, response.text)

In [34]:
# Predefined mapping of body part names to muscle IDs
muscle_name_to_id = {
    "chest": 4,
    "biceps": 1,
    "arms":1,
    "triceps": 5,
    "legs": 8,
    "back": 12,
    "shoulders": 2,
    "abs": 6,
    "calves": 7,
    'Quads': 10,
    'Lats': 12,
    'Hamstrings': 11,
    'Glutes': 8,
}

def get_exercises_by_body_part(body_part_name):
    """
    Fetch exercises based on a body part name.

    Args:
        body_part_name (str): Name of the body part (e.g., "chest").

    Returns:
        List[Dict]: List of exercises for the specified body part.
    """
    muscle_id = muscle_name_to_id.get(body_part_name.lower())
    if not muscle_id:
        print(f"No muscle ID found for body part: {body_part_name}")
        return []

    # API request to fetch exercises
    base_url = "https://wger.de/api/v2/exercise/"
    params = {
        "muscles": muscle_id,
        "language": 2 # English results
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        exercises = response.json().get("results", [])
        return exercises
    except requests.exceptions.RequestException as e:
        print(f"Error fetching exercises for {body_part_name}: {e}")
        return []

In [36]:
#remove html tags
def remove_html_tags(text):
    clean_text = re.sub(r'<.*?>', '', text)  # Remove everything between < and >
    return clean_text

In [38]:
#summarizing the description
def summarize_text(text, max_length=None, min_length=None):

    """
    Summarizes the input text.

    Args:
    text (str): The text to summarize.
    max_length (int): Maximum length of the summary (optional).
    min_length (int): Minimum length of the summary (optional).

    Returns:
    str: The summarized text.
    """

    if not text.strip():
        # Instead of raising ValueError, return an empty string or a message
        return "No description available." # Or ""

    summarizer = pipeline("summarization")
    input_length = len(text.split())

    # Ensure max_length and min_length have valid values
    if max_length is None:
        max_length = max(30, min(130, int(0.3 * input_length)))  # Default: 30% of input length, capped at 130
    if min_length is None:
        min_length = max(10, min(max_length - 10, int(0.1 * input_length)))  # Default: 10% of input length

    # Ensure max_length is greater than min_length
    if max_length <= min_length:
        max_length = min_length + 10

    summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    return summary[0]['summary_text']

In [40]:
#Main method for the workout_plan
def fitness_goals(text,ner):
    text = text.lower()

    final_list = []
    body_parts = extract_body_parts(text,ner)
    if body_parts:
        
        for i in body_parts:
            final_list.append(i[0])
        # print('Body parts identified are:', ','.join(final_list))
        # print("Workout Plan Summary".center(60, "="))
        for part in final_list:
            exercises = get_exercises_by_body_part(part)
            for exercise in exercises[:2]:
                exercise_details = (f"Exercise is {remove_html_tags(exercise['name'])} and here is its brief description: {remove_html_tags(exercise['description'])}"
    )
        return exercise_details
    else:
        return None



In [42]:
# #calling the main method to retrieve the work_out plans
fitness_goals('My are getting bigger from regular rows',ner_body)

NameError: name 'ner_body' is not defined

# **Intent Classification**

In [46]:
def call_bmi():
    try:
        # Get user input for weight and height
        age = int(input("Enter your age (in years): "))
        weight = float(input("Enter your weight (in kilograms): "))
        height = float(input("Enter your height (in meters): "))
        
        # Validate inputs
        if weight <= 0 or height <= 0 or age <= 0:
            return "Invalid input! Age, weight, and height must be positive values."

        # Calculate BMI
        bmi = weight / (height ** 2)

        # Categorize BMI based on standard categories
        if bmi < 18.5:
            category = "Underweight"
        elif 18.5 <= bmi < 24.9:
            category = "Normal weight"
        elif 25 <= bmi < 29.9:
            category = "Overweight"
        else:
            category = "Obesity"

        # Return formatted response
        print(f"Your BMI is {bmi:.2f}, which falls under the category: {category}.")
        return f"Your BMI is {bmi:.2f}, which falls under the category: {category}."
    
    except ValueError:
        return "Invalid input! Please enter numeric values for age, weight, and height."


In [48]:
# NER FOR FOOD
#Loading the spaCy model
nlp_spacy_model = spacy.load("en_core_web_sm")
#Calling the method to add the food label to the Entity Ruler
add_fooditems(nlp_spacy_model)
ner_body = create_fitness_body_parts_ner()

In [51]:
## FINAL METHOD TO FOLLOW gym reccomendation 

In [53]:
# How do I stay motivated 70 25 0.127 1.5 exit


In [ ]:
from flask import Flask, request, render_template
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import logging

# Suppress Transformers warnings
logging.getLogger("transformers").setLevel(logging.ERROR)

# Initialize Flask app
app = Flask(__name__)

# Global variable to store chat history
chat_history = []

# Load fine-tuned GPT-2 model and tokenizer
model_path = "C:\\Users\\ASUS\\Dropbox\\PC\\Downloads\\AIML-lab\\Capstone-project\\fitness_gpt2_model\\fitness_gpt2_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

# Load intents and prepare training data
with open("chat_data.json", "r") as file:
    intents_data = json.load(file)

patterns, tags, responses = [], [], {}
for intent in intents_data["intents"]:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tag"])
    responses[intent["tag"]] = intent["responses"]

# Prepare data for intent classification
df = pd.DataFrame({"patterns": patterns, "tags": tags})
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["patterns"])
encoder = LabelEncoder()
y = encoder.fit_transform(df["tags"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Preprocess user input
def preprocess_input(input_text):
    return input_text.strip().lower()

# Predict intent with confidence scores
def predict_intent(input_text):
    input_vector = vectorizer.transform([input_text])
    prediction = knn.predict(input_vector)
    intent_tag = encoder.inverse_transform(prediction)[0]
    confidence_scores = knn.predict_proba(input_vector)[0]
    max_confidence = max(confidence_scores)
    return intent_tag, max_confidence

# Function to preprocess GPT-2 response
def preprocess_response(response):
    if "Answer:" in response:
        response = response.split("Answer:")[-1].strip()
    if "\n" in response:
        response = response.split("\n")[0].strip()
    if "." in response:
        response = response.split(".")[0].strip() + "."
    return response.strip()

# Function to generate fallback response using GPT-2
def generate_response(question, max_length=50):
    prompt = f"chatbot: {question}"
    ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        top_k=80,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return preprocess_response(response)

# Flask Routes
@app.route("/", methods=["GET", "POST"])
def home():
    global chat_history

    if request.method == "POST":
        user_input = request.form.get("user_input", "")
        user_input = preprocess_input(user_input)

        # Add user input to chat history
        chat_history.append({"sender": "user", "message": user_input})

        # Predict intent and confidence
        intent_tag, max_confidence = predict_intent(user_input)

        if max_confidence < 0.35:  # Confidence threshold
            gpt_response = generate_response(user_input)
        else:
            if intent_tag == "a_gym_recommendation":
                gpt_response = "Please provide your age, weight, and height for BMI calculation."
            elif intent_tag in responses:
                gpt_response = responses[intent_tag][0]
            else:
                gpt_response = generate_response(user_input)

        # Add bot response to chat history
        chat_history.append({"sender": "bot", "message": gpt_response})
        return render_template("index.html", messages=chat_history)

    return render_template("index.html", messages=chat_history)

# Run Flask App
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Dec/2024 19:01:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2024 19:02:47] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2024 19:05:30] "POST / HTTP/1.1" 200 -


Chatbot: Hello! How can I assist you today?
User:  hi
Chatbot (Predefined Response): Hello! How can I assist you with your fitness goals today?
User:  I need gym advice
Chatbot (Predefined Response): Sure! Please provide your age, weight, and height to get a personalized gym recommendation.
User:  calory in beef
Chatbot: The item is a beef. A 100g serving contains: 291.9 calories, 19.7g of total fat, including 7.8g of saturated fat, 26.6g of protein 63mg of sodium, 206mg of potassium, 87mg of cholesterol, 0.0g of total carbohydrates, with 0.0g of fiber and 0.0g of sugar
Chatbot: Is there anything else I can help with?
User:  i want to build biceps
Chatbot: Exercise is Aperturas en polea and here is its brief description: Este ejercicio trabaja y tensa todo el músculo del pecho. Este entrenamiento de pecho es bueno, pero el equipo generalmente solo se encuentra en gimnasios públicos.
Chatbot: Is there anything else I can help with?
User:  I need a balanced diet
Debug: Low confidence (0.33). Sending to fine-tuned GPT-2.
Chatbot (Fine-Tuned GPT-2 Response): Answer concisely: need balanced diet information to assist you in effectively ordering food online
User:  How can I improve my form
Chatbot (Predefined Response): Hello! How can I assist you with your fitness goals today?
User:  How do I increase my bench press weight
Debug: Low confidence (0.33). Sending to fine-tuned GPT-2.
Chatbot (Fine-Tuned GPT-2 Response): Answer concisely: increase bench press weight for your upcoming event at our coffee shop please adhere to the following steps 1 access our website at websiteurl 2 go to the cateringsection 3 choose the catering package that aligns with your specific needs 4 include your